# Chapter 2. Working with Data via SQLAlchemy Core

## 2.1 Inserting Data

In [1]:
from datetime import datetime
import os

In [2]:
from sqlalchemy import (
    Column, DateTime, ForeignKey, Integer, MetaData, Numeric, String, Table, create_engine
)

In [3]:
metadata = MetaData()

cookies = Table(
    'cookies', metadata,
    Column('cookie_id', Integer(), primary_key=True),
    Column('cookie_name', String(50), index=True),
    Column('cookie_recipe_url', String(255)),
    Column('cookie_sku', String(55)),
    Column('quantity', Integer()),
    Column('unit_cost', Numeric(12, 2)),
    Column('created_on', DateTime(), default=datetime.now),
    Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
)

users = Table(
    'users', metadata,
    Column('user_id', Integer(), primary_key=True),
    Column('customer_number', Integer(), autoincrement=True),
    Column('username', String(15), nullable=False, unique=True),
    Column('email_address', String(255), nullable=False),
    Column('phone', String(20), nullable=False),
    Column('password', String(25), nullable=False),
    Column('created_on', DateTime(), default=datetime.now),
    Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
)

orders = Table(
    'orders',
    metadata,
    Column('order_id', Integer(), primary_key=True),
    Column('user_id', ForeignKey('users.user_id')),
    Column('created_on', DateTime(), default=datetime.now),
    Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
)

line_items = Table(
    'line_items',
    metadata,
    Column('line_items_id', Integer(), primary_key=True),
    Column('order_id', ForeignKey('orders.order_id')),
    Column('cookie_id', ForeignKey('cookies.cookie_id')),
    Column('quantity', Integer()),
    Column('extended_cost', Numeric(12, 2))
)

In [4]:
current_folder = os.getcwd()
engine = create_engine('sqlite:///' + os.path.join(current_folder, 'cookies.db'))
metadata.create_all(engine)

Example 2-1. Inserting data as a method

In [5]:
new_cookie = cookies.insert().values(
    cookie_name="chocolate chip",
    cookie_recipe_url="http://some.aweso.me/cookie/recipe.html",
    cookie_sku="CC01",
    quantity="12",
    unit_cost="0.50"
)

In [6]:
print(str(new_cookie))

INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost, created_on, updated_on) VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost, :created_on, :updated_on)


In [7]:
new_cookie.compile().params

{'cookie_name': 'chocolate chip',
 'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe.html',
 'cookie_sku': 'CC01',
 'quantity': '12',
 'unit_cost': '0.50',
 'created_on': None,
 'updated_on': None}

In [8]:
with engine.begin() as connection:
    result = connection.execute(new_cookie)
    print(result.inserted_primary_key)

(1,)


**Example 2-4. Values in execute statement**

In [9]:
insert_statement = cookies.insert()

In [10]:
# result = connection.execute(
#     insert_statement,
#     cookie_name='dark chocolate chip',
#     cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
#     cookie_sku='CC02',
#     quantity='1',
#     unit_cost='0.75'
# )

TypeError: Connection.execute() got an unexpected keyword argument 'cookie_name'

In [11]:
# with engine.begin() as connection:
#     result = connection.execute(
#         insert_statement,
#         cookie_name='dark chocolate chip',
#         cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
#         cookie_sku='CC02',
#         quantity='1',
#         unit_cost='0.75'
#     )
    
#     print(result.inserted_primary_key)

TypeError: Connection.execute() got an unexpected keyword argument 'cookie_name'

In [12]:
with engine.begin() as connection:
    result = connection.execute(
        insert_statement, {
            'cookie_name': 'dark chocolate chip',
            'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe_dark.html',
            'cookie_sku': 'CC02',
            'quantity': '1',
            'unit_cost': '0.75'
        }
    )
    
    print(result.inserted_primary_key)

(2,)


Example 2-5. Multiple inserts

In [13]:
inventory_list = [
    {
    'cookie_name': 'peanut butter',
    'cookie_recipe_url': 'http://some.aweso.me/cookie/peanut.html',
    'cookie_sku': 'PB01',
    'quantity': '24',
    'unit_cost': '0.25'
    },
    {
    'cookie_name': 'oatmeal raisin',
    'cookie_recipe_url': 'http://some.okay.me/cookie/raisin.html',
    'cookie_sku': 'EWW01',
    'quantity': '100',
    'unit_cost': '1.00'
    }
]

In [14]:
with engine.begin() as connection:
    result = connection.execute(insert_statement, inventory_list)
    print(result.rowcount)

2


## 2.2 2. Querying Data

In [16]:
from sqlalchemy.sql import select

In [19]:
selection = select(cookies)

with engine.begin() as connection:
    results = connection.execute(selection)
    for result in results:
        print(result)

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50'), datetime.datetime(2024, 9, 13, 11, 42, 17, 395354), datetime.datetime(2024, 9, 13, 11, 42, 17, 395354))
(2, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, Decimal('0.75'), datetime.datetime(2024, 9, 13, 11, 55, 2, 636436), datetime.datetime(2024, 9, 13, 11, 55, 2, 636436))
(3, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25'), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962))
(4, 'oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, Decimal('1.00'), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962))


**Example 2-7. Simple select method**

In [26]:
selection = cookies.select()

with engine.begin() as connection:
    results = connection.execute(selection)
    for result in results:
        print(result)

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50'), datetime.datetime(2024, 9, 13, 11, 42, 17, 395354), datetime.datetime(2024, 9, 13, 11, 42, 17, 395354))
(2, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, Decimal('0.75'), datetime.datetime(2024, 9, 13, 11, 55, 2, 636436), datetime.datetime(2024, 9, 13, 11, 55, 2, 636436))
(3, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25'), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962))
(4, 'oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, Decimal('1.00'), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962), datetime.datetime(2024, 9, 13, 11, 58, 14, 394962))


**Example 2-10. Select only cookie_name and quantity**

In [27]:
selection = select(cookies.c.cookie_name, cookies.c.quantity)

with engine.connect() as connection:
    results = connection.execute(selection)
    
    print(results.keys())

    result = results.first()
    
    print(result)

RMKeyView(['cookie_name', 'quantity'])
('chocolate chip', 12)


### 2.2.3 Ordering

**Example 2-11. Ordering by quantity ascending**

In [28]:
selection = select(cookies.c.cookie_name, cookies.c.quantity)
selection = selection.order_by(cookies.c.quantity)

with engine.connect() as connection:
    results = connection.execute(selection)
    
    for result in results:
        print('{} - {}'.format(result.quantity, result.cookie_name))

1 - dark chocolate chip
12 - chocolate chip
24 - peanut butter
100 - oatmeal raisin


**Example 2-12. Order by quantity descending**

In [29]:
from sqlalchemy import desc

selection = select(cookies.c.cookie_name, cookies.c.quantity)
selection = selection.order_by(desc(cookies.c.quantity))

with engine.connect() as connection:
    results = connection.execute(selection)
    
    for result in results:
        print('{} - {}'.format(result.quantity, result.cookie_name))

100 - oatmeal raisin
24 - peanut butter
12 - chocolate chip
1 - dark chocolate chip
